In [1]:
import os
from openai import OpenAI, AsyncOpenAI
from dotenv import load_dotenv

# access API_KEYS
load_dotenv()
client = OpenAI(
    api_key=os.getenv('OPENAI_API_KEY'),
)

## 1. Working with GPT_API:

### System Content:
- Indicate the behaviour for genearating a respones based on the problem to solve
- Indicating the lower and upper threshold for each varaible


In [51]:
system_content = """
You are tasked with describing clouds in relation to different weather phenomena, such as rain, imminent rain, cyclones, and anticyclones. 
Do not mention any name of the following varaibles: Cloud Base, High Cloud, Low Cloud, Medium Cloud, Total Cloud

Here are the guidelines for each variable:\n\n1. 
**Cloud Base**: Describe the altitude and its implication for weather conditions.\n    
    - Example: 'Indicates the potential for rain.'\n\n. 
**High Cloud**: Describe the presence and type of high-altitude clouds, indicating weather patterns.\n    
    - Example: 'High cirrus clouds are present, suggesting fair weather.'\n\n. 
**Low Cloud**: Focus on the type and density of low-altitude clouds and their weather implications.\n    
    - Example: 'Indicates heavy rain.'\n\n. 
**Medium Cloud**: Describe the mid-level clouds and what they indicate about the weather.\n    
    - Example: 'Suggest possible light showers.'\n\n. 
**Total Cloud**: Provide an overall assessment of cloud coverage and its impact on weather.\n    
    - Example: 'The sky is completely overcast, likely leading to continuous rain.'

### \n\nThresholds of the varaibles:\n\n. 
"Cloud Base": {
    "0-2000": "Low altitude clouds, indicating the potential for rain.",
    "2001-4000": "Moderate altitude, suggesting stable weather conditions.",
    "4001-6000": "Relatively high clouds, typically indicating fair weather, could indicate cyclones",
    "6000+": "The cloud base is very high, indicating clear and stable weather with no immediate precipitation.Typical of an anticyclone."
  },
"High Cloud": {
    "0.0-0.25": "Indicating clear skies, potential for clear weather",
    "0.26-0.50": "Suggesting fair weather with some incoming rain.",
    "0.51-0.75": "Indicating possible changes in weather.",
    "0.76-1": "High cloud cover is dense, suggesting an approaching front or significant weather changes, associated with an anticyclone."
  },
"Low Cloud": {
    "0.0-0.25": "Few to no low clouds, indicating clear or fair weather.",
    "0.26-0.50": "Moderate presence of low clouds, suggesting possible light showers.",
    "0.51-0.75": "Significant presence of low clouds, indicating potential for overcast conditions with rain or thunderstorms.",
    "0.76-1": "Dense low cloud cover, suggesting overcast conditions with heavy rain or cyclonic activity."
  },
"Medium Cloud": {
    "0.0-0.25": "Few to no medium clouds, indicating stable weather.",
    "0.26-0.50": "Moderate presence of medium clouds, suggesting potential for light showers.",
    "0.51-0.75": "Significant presence of medium clouds, indicating changes in weather conditions. Indicating rain or thunderstorms.",
    "0.76-1": "Dense medium cloud cover, suggesting potential for continuous rain or storms with an unstable atmosphere."
  },
"Total Cloud": {
    "0.0-0.25": "Clear sky with minimal cloud cover, indicating no significant weather activity.",
    "0.26-0.50": "Partly cloudy sky, suggesting fair weather with some cloud presence.",
    "0.51-0.75": "Mostly cloudy sky, indicating overcast conditions with potential for light rain.",
    "0.76-1": "Completely overcast sky, likely leading to continuous rain or cyclonic activity."
  }
    
\n\nHere are sample descriptions for specific weather phenomena:\n\n 
- **Raining**: 'Low, dense clouds dominate the sky, indicating heavy rain with a low cloud base. Medium clouds add to the overcast conditions.'\n-
- **Going to Rain**: 'The cloud base is lowering with increasing total cloud coverage. Medium clouds are thickening, suggesting imminent rain.'\n- 
- **Cyclone**: 'Total cloud coverage with a mix of high cirrus and low dense clouds, indicating the presence of a cyclone. The cloud base is significantly low.'\n- 
- **Anticyclone**: 'High cirrus clouds are scattered with minimal total cloud coverage, indicating fair weather associated with an anticyclone.'\n\n

### Weather Predictions Based on Cloud Data:\n\n1. 
**Precipitation**:\n    
    - **Rain**: Presence of low, dense, and thick clouds like nimbostratus or cumulonimbus suggests rain or thunderstorms.\n    
    - **Snow**: Similar to rain, but typically observed in colder conditions. Thick, low clouds can indicate snowfall.\n\n. 
**Clear Weather**:\n    
    - **Fair Weather**: High, wispy clouds like cirrus indicate fair weather. Scattered cumulus clouds can also suggest clear skies.\n\n. 
**Storms and Severe Weather**:\n    
    - **Thunderstorms**: Tall, towering cumulonimbus clouds are a strong indicator of thunderstorms, possibly with lightning, heavy rain, and hail.\n    
    - **Cyclones**: Total cloud coverage with organized patterns, including low dense clouds and high cirrus clouds, can indicate cyclonic activity.\n\n. 
**Imminent Weather Changes**:\n    
    - **Incoming Rain**: Lowering cloud base with increasing medium and low cloud coverage can suggest imminent rain.\n   
    - **Fronts and Changes in Weather**: The presence of certain clouds like altostratus can indicate a warm front, which often precedes longer periods of precipitation.\n\n. 
**Atmospheric Stability**:\n    
    - **Stable Atmosphere**: High, thin clouds like cirrostratus or cirrocumulus indicate a stable atmosphere, often leading to calm weather.\n    
    - **Unstable Atmosphere**: Presence of towering cumulus or cumulonimbus clouds indicates an unstable atmosphere, likely leading to storms.\n\n
    
By following these guidelines, you can effectively describe the observed clouds, infer weather conditions, and make predictions concisely.
    
Provide your descriptions in a maximum of four sentences.
Generate a description of the weather without mentionning any of the previous variables
""" 

#Maybe trying AWS for the LLama part for taking expirience in this topics, maybe it's useful 

### User Content:
- Indicate the promnt to generate a description for the different varaibles of the clouds in this case

In [55]:
user_content = {
                  "cloud_base"  : 2000,
                  "cloud_high"  : 0.20,
                  "cloud_low"   : 0.9,
                  "cloud_mid"   : 1,
                  "cloud_total" : 0.80
                }

prompt = f"""
            The weather data is as follows:
            Cloud Base: {user_content['cloud_base']} meters
            High Cloud: {user_content['cloud_high']*100}%
            Low Cloud:  {user_content['cloud_low']*100}%
            Medium Cloud: {user_content['cloud_mid']*100}%
            Total Cloud: {user_content['cloud_total']*100}%

            Based on this data, provide an analysis of the cloud coverage and any potential implications for weather conditions.
            Do not mention in the prompt: Cloud Base, High Cloud, Low Cloud, Medium Cloud, Total Cloud
        """

In [56]:
client = AsyncOpenAI()
completion = await client.chat.completions.create(
    model = "gpt-4o", 
    messages = [
    {
        "role"   : "system",
        "content": system_content
    },
    
    {
        "role"   : "user",
        "content": prompt
    }
  ],
  temperature = 0.7
)

In [57]:
print(completion.choices[0].message)

ChatCompletionMessage(content='The altitude of the clouds is moderate, suggesting stable weather conditions. There are few high clouds, indicating clear skies at higher altitudes. However, the sky is dominated by dense low clouds and thick medium clouds, creating an overcast situation. This combination is likely to lead to heavy rain, given the significant total cloud coverage.', role='assistant', function_call=None, tool_calls=None)


# 2. Fine Tunning GPT_API

### Prepare the data

In [9]:
data = [
          {"prompt": "Pregunta: ¿Cómo estás?\n", "completion": "Respuesta: Estoy bien, gracias.\n"},
          {"prompt": "Pregunta: ¿Cuál es tu nombre?\n", "completion": "Respuesta: Soy un modelo de lenguaje GPT.\n"}
       ]

### Upload the file to OpenAI

In [ ]:
response = openai.File.create(
  file=open("ruta/a/tu/archivo.jsonl"),
  purpose='fine-tune'
)

file_id = response['id']

### Init the fine tunning

In [ ]:
response = openai.FineTune.create(
  training_file=file_id,
  model="gpt-3.5-turbo"
)

fine_tune_id = response['id']

### Monitor the process

In [ ]:
response = openai.FineTune.retrieve(id=fine_tune_id)
status = response['status']
print(f"Estado del fine-tuning: {status}")

# List all fine-tunning works
response = openai.FineTune.list()
for fine_tune in response['data']:
    print(fine_tune)

### Use fine-tunning model

In [ ]:
response = openai.Completion.create(
  model="davinci:ft-usuario-empresa-fecha",  # Reemplaza con el ID de tu modelo afinado
  prompt="Pregunta: ¿Cómo estás?\n",
  max_tokens=50
)

response.choices[0].text